Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [1]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import pandas as pd

In [4]:
%%time

transactions = pd.read_parquet('../input/warmup/transactions_train.parquet')
customers = pd.read_parquet('../input/warmup/customers.parquet')
articles = pd.read_parquet('../input/warmup/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: user 3.26 s, sys: 2.08 s, total: 5.34 s
Wall time: 6.26 s


In [5]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [11]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 25.3 s, sys: 427 ms, total: 25.8 s
Wall time: 25.5 s


In [12]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [13]:
c2weeks

customer_id
28847241659200          [95, 96, 101, 102]
41318098387474                        [98]
116809474287335                 [101, 103]
200292573348128          [95, 96, 99, 102]
248294615847351                       [96]
                               ...        
18446624797007271432                  [95]
18446630855572834764                 [103]
18446662237889060501                 [100]
18446705133201055310                 [102]
18446737527580148316                 [104]
Name: week, Length: 437365, dtype: object

In [14]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 948 ms, sys: 66.9 ms, total: 1.01 s
Wall time: 1.01 s


In [15]:
c2weeks2shifted_weeks[28847241659200]

{95: 96, 96: 101, 101: 102, 102: 105}

In [16]:
candidates_last_purchase = transactions.copy()

In [17]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 28.5 s, sys: 78.5 ms, total: 28.6 s
Wall time: 28.6 s


In [18]:
candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,96
29030504,2020-07-15,272412481300040,816592008,0.016932,1,96
29030505,2020-07-15,272412481300040,621381021,0.033881,1,96
29030506,2020-07-15,272412481300040,817477003,0.025407,1,96
29030507,2020-07-15,272412481300040,899088002,0.025407,1,96
29319533,2020-07-22,272412481300040,885077001,0.008458,1,103
29410772,2020-07-24,272412481300040,850176003,0.029034,2,103
29410773,2020-07-24,272412481300040,875803001,0.064559,2,103
29410774,2020-07-24,272412481300040,892970003,0.020966,2,103
29410775,2020-07-24,272412481300040,854619003,0.020966,2,103


In [19]:
transactions[transactions['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95
29319533,2020-07-22,272412481300040,885077001,0.008458,1,96
29410772,2020-07-24,272412481300040,850176003,0.029034,2,96
29410773,2020-07-24,272412481300040,875803001,0.064559,2,96
29410774,2020-07-24,272412481300040,892970003,0.020966,2,96
29410775,2020-07-24,272412481300040,854619003,0.020966,2,96


### Bestsellers candidates

In [20]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [21]:
mean_price

week  article_id
95    108775015     0.004729
      108775044     0.008458
      110065001     0.006085
      110065002     0.006085
      111565001     0.004288
                      ...   
104   952267001     0.013732
      952938001     0.048651
      953450001     0.016932
      953763001     0.021885
      956217002     0.059068
Name: price, Length: 196880, dtype: float32

In [22]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [23]:
sales

week  article_id
95    760084003      1
      866731001      2
      600886001      3
      706016001      4
      372860002      5
                    ..
104   915529003      8
      915529005      9
      448509014     10
      762846027     11
      714790020     12
Name: bestseller_rank, Length: 120, dtype: int8

In [24]:
sales.loc[95]

article_id
760084003     1
866731001     2
600886001     3
706016001     4
372860002     5
610776002     6
877278002     7
547780003     8
817354001     9
827968001    10
866731003    11
866383006    12
Name: bestseller_rank, dtype: int8

In [25]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [26]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [27]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [28]:
unique_transactions

,t_dat,customer_id,sales_channel_id,week
29030503,2020-07-15,272412481300040,1,95
29064059,2020-07-15,1456826891333599,1,95
29067103,2020-07-15,2133687643102426,2,95
29027487,2020-07-15,6010692573790711,1,95
29046403,2020-07-15,6171059100114610,2,95
...,...,...,...,...
31760188,2020-09-22,18435221511488011015,1,104
31782234,2020-09-22,18436859303155335645,1,104
31787251,2020-09-22,18437941771381362708,2,104
31776022,2020-09-22,18438270306572912089,1,104


In [29]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,95
...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,104


In [30]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [31]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [32]:
test_set_transactions

,t_dat,customer_id,sales_channel_id,week
0,2020-07-15,272412481300040,1,105
1,2020-07-15,1456826891333599,1,105
2,2020-07-15,2133687643102426,2,105
3,2020-07-15,6010692573790711,1,105
4,2020-07-15,6171059100114610,2,105
...,...,...,...,...
437360,2020-09-22,18410229429441241008,2,105
437361,2020-09-22,18417769707947924979,2,105
437362,2020-09-22,18418054986721795659,2,105
437363,2020-09-22,18421175435799911749,2,105


In [33]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [34]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [35]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193
...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,1,105,915529003,0.033439
5248376,2020-09-22,18438270306572912089,1,105,915529005,0.033417
5248377,2020-09-22,18438270306572912089,1,105,448509014,0.041630
5248378,2020-09-22,18438270306572912089,1,105,762846027,0.025005


# Combining transactions and candidates / negative examples

In [36]:
transactions['purchased'] = 1

In [37]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [38]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95,1.0
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95,1.0
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95,1.0
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95,1.0
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95,1.0
...,...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0
5248376,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0
5248377,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0
5248378,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0


In [39]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [40]:
data.purchased.mean()

0.13607582749165664

### Add bestseller information

In [41]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [42]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [43]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

### Features

Feature 1: week_nr (which week out of the 52 weeks in a year is it)

In [44]:
# data.sample(20)

In [45]:
data['week_nr'] = data['t_dat'].dt.isocalendar().week.to_frame().astype("int32")
data.dtypes

t_dat                           datetime64[ns]
customer_id                             uint64
article_id                               int32
price                                  float32
sales_channel_id                          int8
week                                     int64
purchased                              float64
bestseller_rank                        float64
product_code                             int32
prod_name                                int32
product_type_no                          int32
product_type_name                        int16
product_group_name                        int8
graphical_appearance_no                  int32
graphical_appearance_name                 int8
colour_group_code                        int32
colour_group_name                         int8
perceived_colour_value_id                int32
perceived_colour_value_name               int8
perceived_colour_master_id               int32
perceived_colour_master_name              int8
department_no

In [46]:
data.sort_values(by=['t_dat'])
group = data.groupby('customer_id')
group.head(30)

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,...,1018,12,9213,1,1,0,1,25,63947,30
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,...,1005,0,7303,1,1,0,1,25,63947,30
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,...,1002,2,1227,1,1,0,1,25,63947,30
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,...,1018,12,52,1,1,0,1,25,63947,30
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,...,1016,11,95,-1,-1,0,0,46,8666,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17991757,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0,8.0,915529,7046,...,1003,3,10909,1,1,0,1,71,116920,39
17991758,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0,9.0,915529,7046,...,1003,3,10909,1,1,0,1,71,116920,39
17991759,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0,10.0,448509,259,...,1009,5,255,1,1,0,1,71,116920,39
17991760,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0,11.0,762846,472,...,1010,6,492,1,1,0,1,71,116920,39


In [47]:
group = data.sort_values(['t_dat'], ascending=True).groupby('customer_id')
data['days_since_last_purchase'] = group['t_dat'].diff()

In [48]:
data['days_since_last_purchase'] = data['days_since_last_purchase'].dt.days

In [49]:
# print(data['time_since_last_purchase'].value_counts().to_string())

In [50]:
# data['time_since_last_purchase'].plot.bar()
# plt.show()

In [51]:
data.dtypes
data.head(20)

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr,days_since_last_purchase
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,...,12,9213,1,1,0,1,25,63947,30,0.0
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,...,0,7303,1,1,0,1,25,63947,30,0.0
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,...,2,1227,1,1,0,1,25,63947,30,0.0
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,...,12,52,1,1,0,1,25,63947,30,0.0
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,...,11,95,-1,-1,0,0,46,8666,30,0.0
5,2020-07-22,248294615847351,471714002,0.007847,1,96,1.0,999.0,471714,243,...,16,635,-1,-1,0,0,46,8666,30,0.0
6,2020-07-22,248294615847351,878987003,0.025169,1,96,1.0,999.0,878987,4158,...,16,2139,-1,-1,0,0,46,8666,30,0.0
7,2020-07-22,248294615847351,337991001,0.023593,1,96,1.0,999.0,337991,3818,...,1,778,-1,-1,0,0,46,8666,30,0.0
8,2020-07-22,272412481300040,885077001,0.008458,1,96,1.0,999.0,885077,9486,...,1,10437,1,1,0,1,48,333369,30,0.0
9,2020-07-22,649760207043851,878502001,0.022017,1,96,1.0,999.0,878502,2008,...,0,11805,1,1,0,1,35,38525,30,0.0


Feature 3: image captions

In [54]:
import pandas as pd # data processing
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import os
plt.style.use("ggplot")
pd.set_option('display.max_columns', 200)

In [55]:
!pip install fastparquet
import fastparquet

In [52]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", device_map={"": 0}, torch_dtype=torch.float16
)  # doctest: +IGNORE_RESULT


KeyboardInterrupt



In [ ]:
# get 1000 most popular items
sales = transactions['article_id'].value_counts().rank(method='dense', ascending=False).head(1000).rename('bestseller_rank').astype('int8')

In [ ]:
appearances = transactions['article_id'].value_counts()
appearances.head(1000)

In [ ]:
bestseller_list = sales.index.tolist()
bestseller_list[0]

Now look for the image files of these bestseller items and generate the description

In [ ]:
# Creating Empty DataFrame and Storing it in variable df 
captions = pd.DataFrame(columns=['article_id', 'caption'])

In [ ]:
%%time
# import required module
import os
from PIL import Image
 
# assign directory
directory = '../input/h-and-m-personalized-fashion-recommendations/images'

image_ids = []

# iterate over files in 
# that directory
i=0
limit = 1005
for subfolder in os.scandir(directory):
    for image in os.scandir(subfolder):
        path = image.path
        path_id = int(path[-14:-4])
        # check if it is a bestseller
        if path_id in bestseller_list:
            # check if there is already a caption for it
            if path_id not in captions['article_id'].unique():
                with Image.open(path) as img:
                    # plt.imshow(np.asarray(img))
                    inputs = processor(img, return_tensors="pt").to(device, torch.float16)
                    generated_ids = model.generate(**inputs, min_length=12, max_length=20)
                    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

                i+=1

                captions.loc[len(captions.index)] = [path_id, generated_text]

                if i%100 == 0:
                    print(str(i) + " images")
                    # every 100th item: write captions dataframe to a file (for if it should crash)
                    captions.to_parquet('/kaggle/working/captions' + str(i) + '.parquet')

                if i >= limit:
                    break
                
    if i >= limit:
        break

print(str(i) + " images")
captions

We only get 996 captions, probably because 4 of the popular items had no image.

In [ ]:
captions.to_parquet('/kaggle/working/captionsAll.parquet')

In [56]:
captions = pd.read_parquet('/kaggle/working/captionsAll.parquet')
captions

,article_id,caption
0,578476001,the skinny black pants are made from stretchy ...
1,573085042,the skinny jeans are made from a dark wash den...
2,572797002,a white tank top with a scoop neckline and ruf...
3,579541089,the cropped cardigan sweater is made from a so...
4,575347003,six pairs of black socks on a white surface wi...
...,...,...
991,859125001,a white shirt with long sleeves and a button d...
992,859105002,a pair of shorts in beige with buttons on the ...
993,856840001,a black and white top with a zebra print pattern
994,850259001,a top with lemon print and ruffled sleeves on ...


Now we embed the captions with sentenceTransformer

(I looked at Peter Kirby's code from feature engineering because he did this with the regular descriptions)
link here: https://github.com/LienM/ai-project-23-24/blob/main/PeterKirby/FeatureEngineering.ipynb

In [57]:
!pip install -U sentence-transformers

In [79]:
# use sentencetransform to convert captions into integers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L12-v2')
# model = SentenceTransformer('all-mpnet-base-v2')

In [80]:
#converting to list of descriptions to be passed to encoder
descriptions = captions['caption'].tolist()
#encoding
embeddings = model.encode(descriptions)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [81]:
embeddings.shape

(996, 384)

In [82]:
#PCA so we arent adding 300+ new features - top 2 principle components
from sklearn.decomposition import PCA
caption_PCA = PCA(n_components=2)
principle_components = caption_PCA.fit_transform(embeddings)
principle_components.shape

(996, 2)

In [83]:
captions["pc1"],captions["pc2"] = principle_components[:,0], principle_components[:,1]
captions

,article_id,caption,pc1,pc2
0,578476001,the skinny black pants are made from stretchy ...,0.515713,-0.027661
1,573085042,the skinny jeans are made from a dark wash den...,0.543519,0.184461
2,572797002,a white tank top with a scoop neckline and ruf...,-0.261080,-0.044785
3,579541089,the cropped cardigan sweater is made from a so...,0.078601,0.578728
4,575347003,six pairs of black socks on a white surface wi...,0.234815,-0.052228
...,...,...,...,...
991,859125001,a white shirt with long sleeves and a button d...,-0.260891,0.158512
992,859105002,a pair of shorts in beige with buttons on the ...,0.263938,-0.166268
993,856840001,a black and white top with a zebra print pattern,-0.286877,-0.018786
994,850259001,a top with lemon print and ruffled sleeves on ...,-0.285077,-0.032722


In [ ]:
# captions = captions.set_index('article_id')
# captions

In [6]:
data = pd.read_parquet('/kaggle/working/data_with_captions_and_pc.parquet')

In [7]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr,days_since_last_purchase,caption,pc1,pc2
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,...,1,0,1,25,63947,30,0.0,,NaN,NaN
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,...,1,0,1,25,63947,30,0.0,,NaN,NaN
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,...,1,0,1,25,63947,30,0.0,a white t - shirt with a short sleeve and a sh...,-0.170963,0.125308
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,...,1,0,1,25,63947,30,0.0,,NaN,NaN
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,...,-1,0,0,46,8666,30,0.0,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17991757,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0,8.0,915529,7046,...,1,0,1,71,116920,39,0.0,the cropped sweater is black and has puff slee...,-0.055073,0.306553
17991758,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0,9.0,915529,7046,...,1,0,1,71,116920,39,0.0,the cropped sweater in beige is made from a so...,0.123744,0.565313
17991759,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0,10.0,448509,259,...,1,0,1,71,116920,39,NaN,the jeans are light blue and have a faded wash...,0.561383,0.050970
17991760,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0,11.0,762846,472,...,1,0,1,71,116920,39,0.0,the silk shirt is white and has buttons on the...,-0.131861,0.205245


In [62]:
small_sample = data.head(100)
small_sample

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr,days_since_last_purchase
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,59,20,6,1010016,0,52,16,7,5,4,3,1645,197,1,1,2,2,51,27,1018,12,9213,1,1,0,1,25,63947,30,0.0
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,274,7,1,1010016,0,9,0,4,0,5,0,1641,0,0,0,1,0,18,12,1005,0,7303,1,1,0,1,25,63947,30,0.0
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,255,3,0,1010016,0,10,2,3,2,9,2,1676,10,0,0,1,0,16,30,1002,2,1227,1,1,0,1,25,63947,30,0.0
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,59,20,6,1010016,0,91,30,3,2,19,7,4242,5,7,7,1,0,60,22,1018,12,52,1,1,0,1,25,63947,30,0.0
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,272,0,1,1010023,4,7,4,2,3,12,4,5672,82,2,2,3,3,56,46,1016,11,95,-1,-1,0,0,46,8666,30,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020-07-22,39703554858650166,649440048,0.011847,2,96,1.0,999.0,649440,91,255,3,0,1010008,5,9,0,4,0,5,0,5828,9,2,2,3,3,20,23,1005,0,0,1,1,0,1,20,17024,30,0.0
96,2020-07-22,39703554858650166,585480001,0.016932,2,96,1.0,999.0,585480,11182,306,13,4,1010016,0,9,0,4,0,5,0,3710,107,7,7,1,0,61,5,1017,4,9914,1,1,0,1,20,17024,30,0.0
97,2020-07-22,43585159363207567,868161002,0.033881,1,96,1.0,999.0,868161,18005,258,5,0,1010016,0,51,3,3,2,3,10,1515,3,0,0,1,0,11,7,1010,6,11575,1,1,0,1,45,330344,30,0.0
98,2020-07-22,43585159363207567,841793003,0.027102,1,96,1.0,999.0,841793,7979,253,9,0,1010016,0,10,2,3,2,9,2,1515,3,0,0,1,0,11,7,1010,6,10756,1,1,0,1,45,330344,30,0.0


In [86]:
# join captions with data dataframe
data_captions = data.merge(captions, on='article_id', how='left')
data = data_captions

In [ ]:
# data = data.drop(['caption_y', 'pc1_y', 'pc2_y'], axis=1)
# data

In [66]:
data.caption = data.caption.fillna('')
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,week_nr,days_since_last_purchase,caption,pc1,pc2
0,2020-07-22,200292573348128,880777001,0.025407,2,96,1.0,999.0,880777,10832,59,20,6,1010016,0,52,16,7,5,4,3,1645,197,1,1,2,2,51,27,1018,12,9213,1,1,0,1,25,63947,30,0.0,,NaN,NaN
1,2020-07-22,200292573348128,784332002,0.025407,2,96,1.0,999.0,784332,25312,274,7,1,1010016,0,9,0,4,0,5,0,1641,0,0,0,1,0,18,12,1005,0,7303,1,1,0,1,25,63947,30,0.0,,NaN,NaN
2,2020-07-22,200292573348128,827968001,0.016932,2,96,1.0,10.0,827968,1066,255,3,0,1010016,0,10,2,3,2,9,2,1676,10,0,0,1,0,16,30,1002,2,1227,1,1,0,1,25,63947,30,0.0,a white t - shirt with a short sleeve and a sh...,-0.111380,0.258872
3,2020-07-22,200292573348128,599580086,0.011847,2,96,1.0,999.0,599580,36,59,20,6,1010016,0,91,30,3,2,19,7,4242,5,7,7,1,0,60,22,1018,12,52,1,1,0,1,25,63947,30,0.0,,NaN,NaN
4,2020-07-22,248294615847351,720504008,0.031458,1,96,1.0,999.0,720504,1373,272,0,1,1010023,4,7,4,2,3,12,4,5672,82,2,2,3,3,56,46,1016,11,95,-1,-1,0,0,46,8666,30,0.0,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17991757,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0,8.0,915529,7046,252,2,0,1010016,0,9,0,4,0,5,0,1626,1,0,0,1,0,15,0,1003,3,10909,1,1,0,1,71,116920,39,0.0,the cropped sweater is black and has puff slee...,-0.003042,0.288298
17991758,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0,9.0,915529,7046,252,2,0,1010016,0,13,12,1,1,1,14,1626,1,0,0,1,0,15,0,1003,3,10909,1,1,0,1,71,116920,39,0.0,the cropped sweater in beige is made from a so...,0.004166,0.507646
17991759,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0,10.0,448509,259,272,0,1,1010016,0,72,6,3,2,2,1,1747,37,1,1,2,2,53,1,1009,5,255,1,1,0,1,71,116920,39,NaN,the jeans are light blue and have a faded wash...,0.527024,-0.008028
17991760,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0,11.0,762846,472,259,8,0,1010016,0,13,12,1,1,1,14,1515,3,0,0,1,0,11,7,1010,6,492,1,1,0,1,71,116920,39,0.0,the silk shirt is white and has buttons on the...,-0.084428,0.307945


In [67]:
data.to_parquet('/kaggle/working/data_with_captions_and_pc2.parquet')

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


__ __ __ __ end of features

In [8]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [9]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [10]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [17]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 
                  'bestseller_rank', 
#                   'week_nr', 'days_since_last_purchase', 
                  'pc1', 'pc2']

In [18]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 272 ms, sys: 220 ms, total: 492 ms
Wall time: 491 ms


# Model training

In [19]:
from lightgbm.sklearn import LGBMRanker

In [20]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [21]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.848850
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.137802
[LightGBM] [Debug] init for col-wise cost 0.237609 seconds, init for row-wise cost 1.526650 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.785260 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1588
[LightGBM] [Info] Number of data points in the train set: 11381612, number of used features: 20
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
CPU times: user 29.3 s, sys: 1.5 s, total: 30.8 s
Wall time: 9.64 s


In [22]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9990318212758316
age 0.0002460251482023409
article_id 0.00015763178028315822
pc1 9.430168685861948e-05
department_no 8.286781958967097e-05
garment_group_no 7.251139294084947e-05
section_no 7.076458287245472e-05
postal_code 6.025087893001724e-05
club_member_status 5.3436064455705104e-05
Active 4.008294375688286e-05
product_type_no 3.6667939765317e-05
pc2 3.5878441492765026e-05
perceived_colour_value_id 1.776004502062487e-05
FN 0.0
fashion_news_frequency 0.0
index_group_no 0.0
index_code 0.0
perceived_colour_master_id 0.0
colour_group_code 0.0
graphical_appearance_no 0.0


# Calculate predictions

In [23]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


NameError: name 'bestsellers_previous_week' is not defined

# Create submission

In [82]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [83]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 5.74 s, sys: 190 ms, total: 5.93 s
Wall time: 5.92 s


In [84]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [85]:
sub_name = 'basic_model_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)